In [1]:
from nlp_toolkit.functions import LlamaFunction, FuncArg

def get_person_job_info(name:str):
    """Get job title of a person with name"""
    
    config = {
        "wanghuan":"engineer",
        "zhangsan":"teacher",
        "lisi":"doctor",
        "wangwu":"farmer",
    }
    
    return config.get(name, "no such person")

func = LlamaFunction(get_person_job_info, 
                     name = FuncArg(description = "person name in english, lower case string, no space, if chinese encountered, use Chinese pinyin string instead")
                     )

In [2]:
import json
from llamaapi import LlamaAPI

from dotenv import load_dotenv
import os

def ask_job(name):

    api_request_json = {
        "messages": [
            {"role": "user", "content": f"What is the job title of {name}?"},
        ],

        # 可以调用的function(s)
        "functions": [
            func.tool_spec
        ],
        
        "stream": False, 
        
        "function_call": func.name,
        
        "model":"llama-70b-chat"
    }


    load_dotenv()
    LLAMA_API_KEY = os.getenv("LLAMA_API_KEY")

    # Initialize the SDK
    llama = LlamaAPI(LLAMA_API_KEY)

    # Execute the Request
    response = llama.run(api_request_json)
    print(json.dumps(response.json(), indent=2))

    f_resp = response.json()['choices'][0]['message']['function_call']
    return func(**f_resp['arguments'])

In [3]:
ask_job("wanghuan")

{
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_person_job_info",
          "arguments": {
            "name": "wanghuan"
          }
        }
      },
      "finish_reason": "function_call"
    }
  ]
}


'engineer'

In [28]:
ask_job("zhangsan")

{
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_person_job_info",
          "arguments": {
            "name": "zhangsan"
          }
        }
      },
      "finish_reason": "function_call"
    }
  ]
}


'teacher'

In [30]:
ask_job("lisi")

{
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_person_job_info",
          "arguments": {
            "name": "lisi"
          }
        }
      },
      "finish_reason": "function_call"
    }
  ]
}


'doctor'

*Llama生成了函数的输入，可惜的是，一次只能输入一个函数*

In [34]:
func.name

'get_person_job_info'

In [4]:
ask_job('王五')

{
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_person_job_info",
          "arguments": {
            "name": "\u738b\u4e94"
          }
        }
      },
      "finish_reason": "function_call"
    }
  ]
}


'no such person'

如此智障的无法转换拼音，我也是无语